In [2]:
%run constant.ipynb
import requests
import pandas as pd

# 获取历史股价数据的函数
def get_historical_data(symbol, access_token, period_type,period, frequency_type, frequency):
    base_url = 'https://api.schwabapi.com/marketdata/v1/pricehistory'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'symbol': symbol,
        'periodType': period_type,
        'period': period,
        'frequencyType': frequency_type,
        'frequency': frequency
    }

    response = requests.get(base_url, headers=headers, params=params)
    print (frequency_type,response.json())
    if response.status_code == 200:
        # Parse JSON data into a DataFrame
        data = response.json()
        candles = data.get('candles', [])
        df = pd.DataFrame(candles)
        df['Date'] = pd.to_datetime(df['datetime'], unit='ms')  # Convert timestamps to datetime
        df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        return df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

# 计算 EMA 的函数
def calculate_ema(data, window):
    return data['Close'].ewm(span=window, adjust=False).mean()

# 计算并比较 EMA
def compare_emas(symbol, access_token):
    # 获取月线数据
    monthly_data = get_historical_data(symbol, access_token, period_type='year', period=10, frequency_type='monthly', frequency=1)
    monthly_ema = calculate_ema(monthly_data, 169).iloc[-1]

    # 获取周线数据
    weekly_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='weekly', frequency=1)
    weekly_ema = calculate_ema(weekly_data, 169).iloc[-1]

    # 获取日线数据
    daily_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1)
    daily_ema = calculate_ema(daily_data, 169).iloc[-1]

    # 获取最后一天的收盘价
    last_close = daily_data.iloc[-1]['Close']

    # 比较关系
    comparison = {
        'Last_Close': last_close,
        'Monthly_EMA_169': monthly_ema,
        'Weekly_EMA_169': weekly_ema,
        'Daily_EMA_169': daily_ema,
        'Comparison': {
            'Above_Monthly_EMA': last_close > monthly_ema,
            'Above_Weekly_EMA': last_close > weekly_ema,
            'Above_Daily_EMA': last_close > daily_ema
        }
    }

    return comparison

# 示例用法
if __name__ == "__main__":
    access_token = get_new_tokens()  # 替换为实际的获取 token 方法
    symbol = 'AAPL'  # 替换为需要分析的股票代码

    result = compare_emas(symbol, access_token)
    print("最后一天股价与EMA 169的关系:")
    print(result)


monthly {'candles': [{'open': 32.335, 'high': 32.57, 'low': 30.4075, 'close': 31.1075, 'volume': 4554567640, 'datetime': 1425189600000}, {'open': 31.2025, 'high': 33.635, 'low': 30.775, 'close': 31.2875, 'volume': 3984542044, 'datetime': 1427864400000}, {'open': 31.5225, 'high': 33.2425, 'low': 30.84, 'close': 32.57, 'volume': 3816608504, 'datetime': 1430456400000}, {'open': 32.7875, 'high': 32.8475, 'low': 31.12, 'close': 31.35625, 'volume': 3514427364, 'datetime': 1433134800000}, {'open': 31.7125, 'high': 33.2425, 'low': 29.805, 'close': 30.325, 'volume': 4233122788, 'datetime': 1435726800000}, {'open': 30.375, 'high': 30.6425, 'low': 23.0, 'close': 28.19, 'volume': 6429149060, 'datetime': 1438405200000}, {'open': 27.5375, 'high': 29.2225, 'low': 26.84, 'close': 27.575, 'volume': 4826189332, 'datetime': 1441083600000}, {'open': 27.2675, 'high': 30.305, 'low': 26.8275, 'close': 29.875, 'volume': 4454815316, 'datetime': 1443675600000}, {'open': 29.9675, 'high': 30.955, 'low': 27.75, 'c

In [9]:
import sqlite3
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
limit=14
# 获取历史股价数据的函数
def get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1):
    base_url = 'https://api.schwabapi.com/marketdata/v1/pricehistory'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'symbol': symbol,
        'periodType': period_type,
        'period': period,
        'frequencyType': frequency_type,
        'frequency': frequency
    }

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        candles = data.get('candles', [])
        df = pd.DataFrame(candles)
        df['Date'] = pd.to_datetime(df['datetime'], unit='ms')  # Convert timestamps to datetime
        df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        return df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

# 计算 EMA 的函数
def calculate_ema(data, window):
    return data['Close'].ewm(span=window, adjust=False).mean()

# 从数据库读取符号（只读取 date 超过 7 天的 symbol）
def get_symbols_from_db(db_path="plan.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 计算 7 天前的日期
    seven_days_ago = (datetime.now() - timedelta(days=limit)).strftime('%Y-%m-%d')
    
    # 只选择 date 超过 7 天的 symbol
    cursor.execute("""
        SELECT symbol FROM EMA169 
        WHERE date < ? OR date IS NULL
    """, (seven_days_ago,))
    
    symbols = [row[0] for row in cursor.fetchall()]
    conn.close()
    return symbols

# 写入数据库并更新日期
def update_ema_in_db(symbol, daily_ema, weekly_ema, monthly_ema, db_path='plan.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 获取当前日期
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # 更新 EMA 值和日期
    cursor.execute("""
        UPDATE EMA169 
        SET daily = ?, weekly = ?, monthly = ?, date = ? 
        WHERE symbol = ?
    """, (daily_ema, weekly_ema, monthly_ema, current_date, symbol))
    
    conn.commit()
    conn.close()

# 计算并更新 EMA
def compare_and_update_emas(access_token, db_path=r"plan.db"):
    symbols = get_symbols_from_db(db_path)
    for symbol in symbols:
        try:
            # 获取月线数据
            monthly_data = get_historical_data(symbol, access_token, period_type='year', period=10, frequency_type='monthly', frequency=1)
            monthly_ema = calculate_ema(monthly_data, 169).iloc[-1]
            time.sleep(10)
            # 获取周线数据
            weekly_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='weekly', frequency=1)
            weekly_ema = calculate_ema(weekly_data, 169).iloc[-1]
            time.sleep(10)
            # 获取日线数据
            daily_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1)
            daily_ema = calculate_ema(daily_data, 169).iloc[-1]
            time.sleep(20)
            # 更新数据库
            update_ema_in_db(symbol, daily_ema, weekly_ema, monthly_ema, db_path)
            print(f"Updated EMA169 for {symbol}: Daily={daily_ema}, Weekly={weekly_ema}, Monthly={monthly_ema}")

        except Exception as e:
            print(f"Failed to update {symbol}: {e}")

# 示例用法
if __name__ == "__main__":
    access_token = get_new_tokens()  # 替换为实际的获取 token 方法
    compare_and_update_emas(access_token)

Updated EMA169 for ADI: Daily=216.2122380044468, Weekly=189.17885545236922, Monthly=127.04795858961077
Updated EMA169 for CSX: Daily=33.720429257596585, Weekly=32.500989891510834, Monthly=23.424385898065246
Updated EMA169 for EQIX: Daily=884.7523992733945, Weekly=786.0027385737507, Monthly=552.5095530959012
Updated EMA169 for CNI: Daily=110.82835794519502, Weekly=114.4614758062016, Monthly=93.66601350125404
Updated EMA169 for GILD: Daily=86.0830367994115, Weekly=77.27223077823035, Monthly=80.6190123420257
Updated EMA169 for DD: Daily=79.43918622142698, Weekly=72.97155333571241, Monthly=72.5733087524109
Updated EMA169 for CRH: Daily=91.26657933140258, Weekly=67.56829776411269, Monthly=44.383357179895036
Updated EMA169 for CCL: Daily=21.8516714071683, Weekly=18.22329181247226, Monthly=33.90367394574974
